In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from multiprocessing import Pool

import numpy as np
import pandas as pd

import kraft

In [ ]:
gene_x_sample = pd.read_csv("../rna/rna.gene_x_cell_line.tsv", sep="\t", index_col=0)

gene_x_sample = gene_x_sample.loc[gene_x_sample.index & kraft.get_gene_symbol()]

In [ ]:
gene_set_genes = kraft.read_gmts(
    (
        "../../gene_set/h.all.v7.1.symbols.gmt",
        "../../gene_set/c1.all.v7.1.symbols.gmt",
        "../../gene_set/c2.all.v7.1.symbols.gmt",
        "../../gene_set/c3.all.v7.1.symbols.gmt",
        "../../gene_set/c5.all.v7.1.symbols.gmt",
        "../../gene_set/c6.all.v7.1.symbols.gmt",
        "../../gene_set/c7.all.v7.1.symbols.gmt",
        "../../gene_set/ipa.gmt",
    )
)

In [ ]:
def function(element_score, set_elements):

    print(element_score.name)

    element_score = pd.Series(
        kraft.normalize(element_score.values, "-0-"),
        index=element_score.index,
        name=element_score.name,
    ).sort_values()

    return tuple(
        kraft.score_set(
            element_score, elements, method=method, sort_element_score=False, plot=False
        )
        for elements in set_elements.values()
    )

In [ ]:
method = "2020"

In [ ]:
pool = Pool(processes=10)

gene_set_x_sample = pd.DataFrame(
    np.asarray(
        pool.starmap(
            function,
            ((gene_score, gene_set_genes) for _, gene_score in gene_x_sample.items()),
        )
    ).T,
    index=gene_set_genes.keys(),
    columns=gene_x_sample.columns,
)

gene_set_x_sample.to_csv("{}.gene_set_x_sample.tsv".format(method), sep="\t")

gene_set_x_sample